https://www.kaggle.com/guntherthepenguin/fastai-based-mixed-input-model-lb-3-8
https://www.kaggle.com/delayedkarma/let-s-add-some-features-for-the-national-holidays

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.lib.deepreload import reload as dreload
import PIL, os, numpy as np, math, collections, threading, json, bcolz, random, scipy, cv2
import pandas as pd, pickle, sys, itertools, string, sys, re, datetime, time, shutil, copy
import seaborn as sns, matplotlib
import IPython, graphviz, sklearn_pandas, sklearn, warnings, pdb
import contextlib
from abc import abstractmethod
from glob import glob, iglob
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from itertools import chain
from functools import partial
from collections import Iterable, Counter, OrderedDict
from isoweek import Week
from pandas_summary import DataFrameSummary
from IPython.lib.display import FileLink
from PIL import Image, ImageEnhance, ImageOps
from sklearn import metrics, ensemble, preprocessing
from operator import itemgetter, attrgetter
from pathlib import Path
from distutils.version import LooseVersion

from matplotlib import pyplot as plt, rcParams, animation
from ipywidgets import interact, interactive, fixed, widgets
matplotlib.rc('animation', html='html5')
np.set_printoptions(precision=5, linewidth=110, suppress=True)



from ipykernel.kernelapp import IPKernelApp
def in_notebook(): return IPKernelApp.initialized()

def in_ipynb():
    try:
        cls = get_ipython().__class__.__name__
        return cls == 'ZMQInteractiveShell'
    except NameError:
        return False

import tqdm as tq
from tqdm import tqdm_notebook, tnrange

def clear_tqdm():
    inst = getattr(tq.tqdm, '_instances', None)
    if not inst: return
    try:
        for i in range(len(inst)): inst.pop().close()
    except Exception:
        pass

if in_notebook():
    def tqdm(*args, **kwargs):
        clear_tqdm()
        return tq.tqdm(*args, file=sys.stdout, **kwargs)
    def trange(*args, **kwargs):
        clear_tqdm()
        return tq.trange(*args, file=sys.stdout, **kwargs)
else:
    from tqdm import tqdm, trange
    tnrange=trange
    tqdm_notebook=tqdm


from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, Imputer, StandardScaler
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.ensemble import forest
from sklearn.tree import export_graphviz


def set_plot_sizes(sml, med, big):
    plt.rc('font', size=sml)          # controls default text sizes
    plt.rc('axes', titlesize=sml)     # fontsize of the axes title
    plt.rc('axes', labelsize=med)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=sml)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=sml)    # fontsize of the tick labels
    plt.rc('legend', fontsize=sml)    # legend fontsize
    plt.rc('figure', titlesize=big)  # fontsize of the figure title

def parallel_trees(m, fn, n_jobs=8):
        return list(ProcessPoolExecutor(n_jobs).map(fn, m.estimators_))

def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    """
    """
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

def combine_date(years, months=1, days=1, weeks=None, hours=None, minutes=None,
              seconds=None, milliseconds=None, microseconds=None, nanoseconds=None):
    years = np.asarray(years) - 1970
    months = np.asarray(months) - 1
    days = np.asarray(days) - 1
    types = ('<M8[Y]', '<m8[M]', '<m8[D]', '<m8[W]', '<m8[h]',
             '<m8[m]', '<m8[s]', '<m8[ms]', '<m8[us]', '<m8[ns]')
    vals = (years, months, days, weeks, hours, minutes, seconds,
            milliseconds, microseconds, nanoseconds)
    return sum(np.asarray(v, dtype=t) for t, v in zip(types, vals)
               if v is not None)

def get_sample(df,n):
    """
    """
    idxs = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idxs].copy()

import workalendar
from workalendar.america import Brazil
cal = Brazil()
from datetime import timedelta
holidays_delta = {}
def get_working_days_delta(begin, end):
    '''
    Get working days between two dates
    '''
    if begin in holidays_delta:
        return holidays_delta[begin]
    
    try:
        
        sign = 1
        if begin > end:
            begin, end = end, begin
            sign = -1
        days = 0
        temp_day = begin
        while temp_day <= end:
            if cal.is_working_day(temp_day):
                days += 1
            temp_day = temp_day + timedelta(days=1)
        delta = days * sign
        holidays_delta[begin] = delta
    except Exception:
        holidays_delta[begin] = 0
    return holidays_delta[begin] 

def add_datepart(df, fldname, drop=True, time=False, errors="raise"):	
    """
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    try:
        df[targ_pre + 'Holiday_Elapsed'] = fld.apply(
            lambda x: get_working_days_delta(x.date(),cal.holidays(int(x.year))[0][0])
        )
    except Exception:
        df[targ_pre + 'Holiday_Elapsed'] = 0
    
    if drop: df.drop(fldname, axis=1, inplace=True)

def is_date(x): return np.issubdtype(x.dtype, np.datetime64)

def train_cats(df):
    """
    """
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

def apply_cats(df, trn):
    """
    """
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = c.astype('category').cat.as_ordered()
            df[n].cat.set_categories(trn[n].cat.categories, ordered=True, inplace=True)

def fix_missing(df, col, name, na_dict):
    """ 
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def numericalize(df, col, name, max_n_cat):
    """ 
    """
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = col.cat.codes+1

def scale_vars(df, mapper):
    warnings.filterwarnings('ignore', category=sklearn.exceptions.DataConversionWarning)
    if mapper is None:
        map_f = [([n],StandardScaler()) for n in df.columns if is_numeric_dtype(df[n])]
        mapper = DataFrameMapper(map_f).fit(df)
    df[mapper.transformed_names_] = mapper.transform(df)
    return mapper

def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    """ 
    """
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    else: df = df.copy()
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    else: na_dict = na_dict.copy()
    na_dict_initial = na_dict.copy()
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if len(na_dict_initial.keys()) > 0:
        df.drop([a + '_na' for a in list(set(na_dict.keys()) - set(na_dict_initial.keys()))], axis=1, inplace=True)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

def get_nn_mappers(df, cat_vars, contin_vars):
    # Replace nulls with 0 for continuous, "" for categorical.
    for v in contin_vars: df[v] = df[v].fillna(df[v].max()+100,)
    for v in cat_vars: df[v].fillna('#NA#', inplace=True)

    # list of tuples, containing variable and instance of a transformer for that variable
    # for categoricals, use LabelEncoder to map to integers. For continuous, standardize
    cat_maps = [(o, LabelEncoder()) for o in cat_vars]
    contin_maps = [([o], StandardScaler()) for o in contin_vars]
    return DataFrameMapper(cat_maps).fit(df), DataFrameMapper(contin_maps).fit(df)

In [ ]:
path='../input/'

In [ ]:
new_transactions = pd.read_csv(f'{path}new_merchant_transactions.csv', parse_dates=['purchase_date'])
new_transactions.head().T

In [ ]:
new_transactions['authorized_flag'] = new_transactions['authorized_flag']=='Y'

In [ ]:
add_datepart(new_transactions, 'purchase_date') 

In [ ]:
new_transactions.head().T

In [ ]:
def aggregate_new_transactions(new_trans):    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'month_lag': ['min', 'max'],
#         'purchase_date': ['max','min'],
        'purchase_Year':['min', 'max'],
        'purchase_Month':['min', 'max'],
        'purchase_Week':['min', 'max'],
        'purchase_Day':['min', 'max'],
        'purchase_Dayofweek':['min', 'max'],
        'purchase_Dayofyear':['min', 'max'],
        'purchase_Is_month_end':['min', 'max'],
        'purchase_Is_month_start':['min', 'max'],
        'purchase_Is_quarter_end':['min', 'max'],
        'purchase_Is_quarter_start':['min', 'max'],
        'purchase_Is_year_end':['min', 'max'],
        'purchase_Is_year_start':['min', 'max'],
        'purchase_Elapsed':['min', 'max'],
        'purchase_Holiday_Elapsed':['nunique'],
        }
    agg_new_trans = new_trans.groupby(['card_id']).agg(agg_func)
#     print(agg_new_trans.columns)
    agg_new_trans.columns = ['new_' + '_'.join(col).strip() 
                           for col in agg_new_trans.columns.values]
    print(agg_new_trans.columns)
    agg_new_trans.reset_index(inplace=True)
    
    df = (new_trans.groupby('card_id')
          .size()
          .reset_index(name='new_transactions_count'))
    
    agg_new_trans = pd.merge(df, agg_new_trans, on='card_id', how='left')
    
    return agg_new_trans

new_trans = aggregate_new_transactions(new_transactions)
new_trans.head().T

In [ ]:
new_trans.head().T

In [ ]:
del new_transactions

In [ ]:
historical_transactions = pd.read_csv('../input/historical_transactions.csv',parse_dates=['purchase_date'])
historical_transactions.head()

In [ ]:
historical_transactions['authorized_flag'] = historical_transactions['authorized_flag']=='Y'

In [ ]:
add_datepart(historical_transactions, 'purchase_date') 

In [ ]:
def aggregate_historical_transactions(history):
    
#     history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
#                                       astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
#         'purchase_date': [np.ptp],
        'month_lag': ['min', 'max'],
#         'purchase_date': ['max','min'],
        'purchase_Year':['min', 'max'],
        'purchase_Month':['min', 'max'],
        'purchase_Week':['min', 'max'],
        'purchase_Day':['min', 'max'],
        'purchase_Dayofweek':['min', 'max'],
        'purchase_Dayofyear':['min', 'max'],
        'purchase_Is_month_end':['min', 'max'],
        'purchase_Is_month_start':['min', 'max'],
        'purchase_Is_quarter_end':['min', 'max'],
        'purchase_Is_quarter_start':['min', 'max'],
        'purchase_Is_year_end':['min', 'max'],
        'purchase_Is_year_start':['min', 'max'],
        'purchase_Elapsed':['min', 'max'],
        'purchase_Holiday_Elapsed':['nunique'],
        }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['hist_' + '_'.join(col).strip() 
                           for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='hist_transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

history = aggregate_historical_transactions(historical_transactions)
history.head().T

In [ ]:
del historical_transactions

In [ ]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    add_datepart(df,'first_active_month')
    return df
train = read_data('../input/train.csv')
test = read_data('../input/test.csv')

target='target'


In [ ]:
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, new_trans, on='card_id', how='left')
test = pd.merge(test, new_trans, on='card_id', how='left')

In [ ]:
train=train.set_index('card_id')
test=test.set_index('card_id')

In [ ]:
train.head().T

In [ ]:
cat_vars = [col  for col in train.columns if('feature' in col or 'first_active_month' in col)]
contin_vars = [col  for col in train.columns if ('feature' not in col and 'first_active_month' not in col) and target not in col]

In [ ]:
train = train[cat_vars+contin_vars+[target]].copy()
n = len(train); n

In [ ]:
test[target] = 0
test = test[cat_vars+contin_vars+[target]].copy()

In [ ]:
for v in cat_vars: train[v] = train[v].astype('category').cat.as_ordered()
apply_cats(test, train)

In [ ]:
for v in contin_vars:
    train[v] = train[v].fillna(0).astype('float32')
    test[v] = test[v].fillna(0).astype('float32')

In [ ]:
df_trn, y, nas, mapper = proc_df(train, target, do_scale=True)

In [ ]:
n=len(df_trn);n

In [ ]:
df_test, _, nas, mapper = proc_df(test, target, do_scale=True, 
                                  mapper=mapper, na_dict=nas)

In [ ]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()



n_valid = int(len(df_trn) * .25)
n_trn = len(df_trn)-n_valid
# raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
draw_tree(m.estimators_[0], df_trn, precision=3)

In [ ]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=150, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
prediction = m.predict(df_test)

In [ ]:
from datetime import datetime
ids = df_test.index.values
vals = prediction
sub = pd.DataFrame({'card_id': ids, 'target': vals})
sub.to_csv("submission-elo-%s.csv"%datetime.now().date(), index = False, header = True)
sub.head()

In [ ]:
np.max(sub), np.min(sub)

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.')